<a href="https://colab.research.google.com/github/btatsis/learn2smell/blob/master/learn2smell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# Taken from AIcrowd github repo
#!git clone https://github.com/AIcrowd/learning-to-smell-starter-kit
#!cd learning-to-smell-starter-kit
#!pip install -r requirements.txt

!rm -fr *

In [54]:
!mkdir data
!git clone https://github.com/btatsis/learn2smell.git  .data/
#!git clone https://github.com/btatsis/learn2smell/blob/master/test.csv data
#!git clone https://github.com/btatsis/learn2smell/blob/master/vocabulary.txt data

Cloning into '.data'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [60]:
!head .data/train.csv

SMILES,SENTENCE
C/C=C/C(=O)C1CCC(C=C1C)(C)C,"fruity,rose"
COC(=O)OC,"fresh,ethereal,fruity"
Cc1cc2c([nH]1)cccc2,"resinous,animalic"
C1CCCCCCCC(=O)CCCCCCC1,"powdery,musk,animalic"
CC(CC(=O)OC1CC2C(C1(C)CC2)(C)C)C,"coniferous,camphor,fruity"
CCC[C@H](CCO)SC,tropicalfruit
CC(C)C(=O)OCc1ccco1,"fruity,coffee"
OC[C@H]1[C@H]2CC[C@H]3[C@@]1(C)CCCC([C@@H]23)(C)C,woody
CCc1ccc(cc1)CC(C=O)(C)C,"clean,fresh"


In [93]:
df = pd.read_csv('.data/train.csv', sep='^([^,]+),', engine='python', usecols=['SMILES', 'SENTENCE'])
df.rename({'SMILES': 'smiles', 'SENTENCE': 'smell'}, axis=1, inplace=True)
df.smell = df.smell.str.replace('"', '')

df_new = df.smell.str.get_dummies(sep=',')

df = pd.concat([df, df.smell.str.get_dummies(sep=',')],axis=1)
df.drop

In [94]:
df.head()

,smiles,smell,alcoholic,aldehydic,alliaceous,almond,ambergris,ambery,ambrette,ammoniac,animalic,anisic,apple,balsamic,banana,berry,blackcurrant,blueberry,body,bread,burnt,butter,cacao,camphor,caramellic,cedar,cheese,chemical,cherry,cinnamon,citrus,clean,clove,coconut,coffee,cognac,coniferous,cooked,cooling,cucumber,...,mint,mushroom,musk,musty,nut,odorless,oily,orange,overripe,pear,pepper,phenolic,plastic,plum,powdery,pungent,rancid,resinous,ripe,roasted,rose,seafood,sharp,smoky,sour,spicy,sulfuric,sweet,syrup,terpenic,tobacco,tropicalfruit,vanilla,vegetable,violetflower,watery,waxy,whiteflower,wine,woody
0,C/C=C/C(=O)C1CCC(C=C1C)(C)C,"fruity,rose",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,COC(=O)OC,"fresh,ethereal,fruity",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Cc1cc2c([nH]1)cccc2,"resinous,animalic",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,C1CCCCCCCC(=O)CCCCCCC1,"powdery,musk,animalic",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,CC(CC(=O)OC1CC2C(C1(C)CC2)(C)C)C,"coniferous,camphor,fruity",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Script for results submission in AIcrowd competition

In [ ]:
#!/usr/bin/env python

import pandas as pd
import json

import os
import random


TRAINING_DATASET_PATH = os.getenv("TRAINING_DATASET_PATH", "data/train.csv")
TEST_DATASET_PATH = os.getenv("TEST_DATASET_PATH", "data/test.csv")

VOCABULARY_PATH = os.getenv("VOCABULARY_PATH", "data/vocabulary.txt")


vocabulary = open(VOCABULARY_PATH).read().split()


test_df = pd.read_csv(TEST_DATASET_PATH)


def generate_sample_submission(df, vocabulary):
    """
    Generates a sample submission file
    by randomly generating 5 random smell sentences 
    of upto 3 random smell words.

    The 5 sentences are to be separated by a ;
    and all the individual smells words in 
    a sentence are to be separated by a ,
    """

    def generate_random_prediction():
        # Generates a single prediction string

        SENTENCES = []
        for _ in range(5):
            WORDS = []
            for _ in range(3):
                WORDS.append(
                    random.choice(vocabulary)
                )
            SENTENCES.append(",".join(sorted(set(WORDS))))
        return ";".join(SENTENCES)
    
    predictions = [generate_random_prediction() for _ in range(df.shape[0])]

    return pd.DataFrame({
        "SMILES" : df.SMILES.tolist(),
        "PREDICTIONS" : predictions
    })

sample_submission_df = generate_sample_submission(test_df, vocabulary)

submission_file_path = "submission.csv"
print("Writing Sample Submission to : ", submission_file_path)
sample_submission_df.to_csv(
    submission_file_path,
    index=False
)